In [1]:
from src import rft_main

import numpy as np


In [2]:
### INPUTS ###
inputs = {
    "model": "CylinderRough",
    "bulk_density": 1310,
    "friction_material": 0.21,
    "firction_surface": 0.4,
    "friction_type": "coefficient",
    "gravity": 9.81,
    "linear_velocity": 0.1,
    "direction_angle_xz_deg": -90,
    "direction_angle_y_deg": -90,
    "rotation": False,
    "angular_velocity": np.array([0, 0, -2 * np.pi]),
    "start_depth": 100,
    "end_depth": 200,
    "step_size": 5,
}


In [3]:
rft_main.run_rft(**inputs)

print("Done!")


NameError: name 'rft_functions' is not defined